<a href="https://colab.research.google.com/github/BlackHawk0/100daysofDjango/blob/main/Copy_of_QBUS2310_2022S1_Assignment2_code_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QBUS2310 Management Science

## Assignment 2
Semester 1, 2022

Please answer all questions in the spaces provided in this notebook. Make sure you fill in any place that says
`YOUR CODE HERE`. Remove the
```Python
raise NotImplementedError()
```
lines.

Before you submit, make sure everything runs as expected. In the menubar, select Kernel$\rightarrow$Restart & Run All.

When you are done, save the file and upload it to Canvas. You should submit one file for the whole assignment.  

__Good luck!__

In [3]:
!pip install gurobipy

     |████████████████████████████████| 11.5 MB 27.2 MB/s 


In [4]:
# Importing packages
## If you are using Google Colab, please remember to comment out the `!pip install gurobipy` line
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime, timedelta
from itertools import combinations

## Question 3

We will implement our tournament elimination model and test it on real data. The following script loads results data for the 2018-2019 NBA regular season into a `pandas.DataFrame` object. The columns are

* `GAME_DATE_EST`: the date of the game in Eastern Standard Time (EST).
* `HOME_TEAM_ID`: the name of the home team.
* `VISITOR_TEAM_ID`: the name of the visitor team.
* `HOME_TEAM_WINS`: 1 if the home team won, 0 if the home team lost (and the visitor team won).

In [4]:
data = pd.read_csv('games_NBA_1819.csv')
data['GAME_DATE_EST'] = pd.to_datetime(data['GAME_DATE_EST'])

In [5]:
### Importing solution file. Do not move or delete this cell.

### Q3(a) Data preparation

(5 points) We need to extract the numbers that we need. Complete the `compute_numbers` function below so that it does the following:

* takes as input the `year`, `month`, `day` of a certain date and `pandas.DataFrame`.
* computes the total wins for each team up to and including the input date. Store this as a dictionary with team name keys and values as the number of wins.
* computes the remaining games for each team pair. Store this as a dictionary with team pair keys and values as the number of games left for that pair.

We have provided an extract of the expected results for two dates to test your function.

**Date 1: 22nd October 2018**

```Python
wins_1, games_to_play_1 = compute_numbers(2018, 10, 22, df=data)
print(wins_1)
```
```
{'Warriors': 3, 'Celtics': 2, 'Clippers': 2, 'Hornets': 2, 'Pistons': 2, ...
```
```Python
print(games_to_play_1)
```
```
{('Warriors', 'Celtics'): 2, ('Warriors', 'Clippers'): 4, ('Warriors', 'Hornets'): 2, ('Warriors', 'Pistons'): 2, ('Warriors', 'Pacers'): 2, ('Warriors', 'Magic'): 2, ('Warriors', 'Suns'): 3, ('Warriors', 'Raptors'): 2, ...
```

**Date 2: 3rd February 2019**

```Python
wins_2, games_to_play_2 = compute_numbers(2019, 2, 3, df=data)
print(wins_2)
```
```
{'Warriors': 37, 'Celtics': 34, 'Clippers': 29, 'Hornets': 26, 'Pistons': 22, ...
```
```Python
print(games_to_play_2)
```
```
{('Warriors', 'Celtics'): 1, ('Warriors', 'Clippers'): 1, ('Warriors', 'Hornets'): 2, ('Warriors', 'Pistons'): 1, ('Warriors', 'Pacers'): 1, ('Warriors', 'Magic'): 1, ('Warriors', 'Suns'): 2, ('Warriors', 'Raptors'): 0, ...
```

In [61]:
def compute_numbers(year, month, day, df=data):

    ## Construct datetime object for easy slicing
    ## (note that the date column of the data has been converted to datetime objects)
    date = datetime(year, month, day)

    ## Slice dataframe into past and future
    past = df[(df.GAME_DATE_EST <= date)]
    future = df[(df.GAME_DATE_EST > date)]

    teams = df.HOME_TEAM_ID.unique()

    wins = {t: 0 for t in teams}
    games_to_play = {p: 0 for p in combinations(teams, 2)}

    ###########################
    ### YOUR CODE HERE ########
    # Compute wins
    for team in teams:
        wins[team] = len(past[(past.HOME_TEAM_ID == team) | (past.VISITOR_TEAM_ID == team) & (past.HOME_TEAM_WINS == 1)])

    # Compute games to play
    for team1, team2 in combinations(teams, 2):
        games_to_play[(team1, team2)] = len(future[((future.HOME_TEAM_ID == team1) & (future.VISITOR_TEAM_ID == team2)) | ((future.HOME_TEAM_ID == team2) & (future.VISITOR_TEAM_ID == team1))])

    return wins, games_to_play


In [7]:
# Assertion cell: Q3a - Please to not edit or delete this cell

In [8]:
# Assertion cell: Q3a - Please to not edit or delete this cell

In [9]:
# Assertion cell: Q3a - Please to not edit or delete this cell

### Q3(b) Implement model

(8 points) Complete the `team_eliminated` function below that takes a team name stored in `team`, and the `wins`, `games_to_play` dictionaries (which are outputs of the `compute_numbers` function above) and checks whether that team is eliminated or not. Your function should return the Boolean `True` if the team is eliminated and `False` if otherwise.

In [82]:
pip install pulp

     |████████████████████████████████| 14.2 MB 5.2 MB/s 


In [86]:
from scipy.optimize import linprog
import pulp as p
def team_eliminated(team, wins, games_to_play):

    ## Compute the best possible win total for the given team
    W = wins[team] + sum([games_to_play[p] for p in games_to_play if p[0] == team or p[1] == team])

    ## Construct sets which don't contain the team
    teamlist = [t for t in wins if t != team]
    pairs = [p for p in games_to_play if p[0] != team and p[1] != team]

    ## Construct the index sets for the x variables
    idx = [(h, a, h) for h, a in pairs] + [(h, a, a) for h, a in pairs]

    ## Boolean variable that you need to modify
    team_eliminated = None

    ###########################
    ### YOUR CODE HERE ########
  # Create LP
    prob = p.LpProblem("NBA", p.LpMaximize)

    # Create LP variables
    lp = p.LpVariable.dict("lp", [(h, a, x) for h, a, x in idx], lowBound=0, cat='Continuous')

    # Objective Function
    prob += p.lpSum([lp[(h, a, x)] for h, a, x in idx]), "Objective Function"

    # Constraints
    for x in teamlist:
        prob += p.lpSum([lp[(h, a, x)] for (h, a, _x) in lp if _x == x]) <= W - wins[x], ""

    # Solve LP
    prob.solve()

    if p.LpStatus[prob.status] == 'Optimal':
        for v in prob.variables():
            h, a, x = v.name.split("_")
            if x == team:
                if v.varValue > 0:
                    team_eliminated = True
                else:
                    team_eliminated = False

    return team_eliminated

The following `which_teams_remaining` function takes the `wins` and `games_to_play` dictionaries, and computes which teams are eliminated and which are remaining, using the `team_eliminated` function above. You don't need to modify this function.

In [84]:
def which_teams_remaining(wins, games_to_play):

    remaining = []
    eliminated = []
    for t in wins:
        elim = team_eliminated(t, wins, games_to_play)
        if (elim):
            eliminated.append(t)
        else:
            remaining.append(t)
    return remaining, eliminated

Here is an example usage of the functions above for checking which teams are remaining on 25th February 2019.

In [87]:
wins, games_to_play = compute_numbers(2019, 2, 25, df=data)

remaining, eliminated = which_teams_remaining(wins, games_to_play)

TypeError: ignored

In [13]:
# Assertion cell: Q3b - Please to not edit or delete this cell

In [14]:
# Assertion cell: Q3b - Please to not edit or delete this cell

### Q3(c) Interpretation questions

Assign your student ID number to the appropriate variable in the following cell and run the subsequent cells to determine your question set.

Before answering the questions, check that the code makes sense by running a few simple examples. E.g., the 2018-19 NBA regular season started on October 16 2018, and ended on April 10 2019. Therefore, trying a date of October 17 should not eliminate anyone, and trying April 10 should eliminate everyone except the teams with the highest wins.

> **Note:** The computation for this question may take some time. Each code cell you submit has a 150-second execution time limit. As a rough guide, it should take around 0.5 seconds to run an instance of `which_teams_remaining`.

In [15]:
# You should assign your student ID number to the variable SID
# To get your question set, run the following cells.
# Make sure this is your SID. We will NOT use this variable when testing as we
# will obtain it externally.
SID = None
###########################
### YOUR CODE HERE ########

SID = 'A0173839W'



In [16]:
def generate_question_set(SID, df=data):
    '''
    This is a function to determine your question set.
    '''
    if len(str(SID)) == 9:
        np.random.seed(int(SID))
        teams = set(df.HOME_TEAM_ID.unique())
        teams_list_1 = list(teams.difference(['Bucks', 'Warriors', 'Raptors']))
        team_elim = np.random.choice(teams_list_1)

        teams_list_2 = list(teams)

        team_1, team_2 = np.random.choice(teams_list_2, size=2, replace=False)

        n_eliminated = np.random.randint(low=5, high=25)
        n_remaining = np.random.randint(low=5, high=25)

        if n_eliminated + n_remaining == 30:
            n_eliminated -= 2
            n_remaining -= 2

        print(f'''You should answer the following questions:
        
1) (3 points) What date are the {team_elim} eliminated? 
    Assign your answer to the variable ans_3c_team_eliminated as a tuple containing the (year, month, day).

2) (3 points) Are the {team_1} eliminated before the {team_2}? 
    If the {team_1} are eliminated before the {team_2}, assign the Boolean True to the variable ans_3c_before.
    Otherwise, if the {team_2} are eliminated before the {team_1}, or if they are eliminated on the same day, 
    assign the Boolean False to the variable ans_3c_before.
    
3) (3 points) What is the first date that at least {n_eliminated} teams are eliminated? 
    Assign your answer to the variable ans_3c_n_eliminated as a tuple containing the (year, month, day).

4) (3 points) What is the last date that there were at least {n_remaining} teams remaining? 
    Assign your answer to the variable ans_3c_n_remaining as a tuple containing the (year, month, day).

Note: The date a team is eliminated is the first date they appear in the eliminated list.
''')
        return 'End of question 3c.'
    else:
        raise ValueError("Not a valid SID!")

In [17]:
print(generate_question_set(SID, df=data))

ValueError: ignored

In [18]:
# Q3c Assign your answers to the following variables.
year, month, date = None, None, None
ans_3c_team_eliminated = (year, month, date)
ans_3c_before = None
ans_3c_n_eliminated = (year, month, date)
ans_3c_n_remaining = (year, month, date)


###########################
### YOUR CODE HERE ########
year, month, date = 2018, 10, 22
ans_3c_team_eliminated = (year, month, date)
ans_3c_before = None
ans_3c_n_eliminated = (year, month, date)
ans_3c_n_remaining = (year, month, date)




In [19]:
# Assertion cell: Q3c - Please to not edit or delete this cell

In [20]:
# Assertion cell: Q3c - Please to not edit or delete this cell

In [21]:
# Assertion cell: Q3c - Please to not edit or delete this cell

In [22]:
# Assertion cell: Q3c - Please to not edit or delete this cell

In [23]:
# Assertion cell: Q3c - Please to not edit or delete this cell

## Question 5

### Q5(c)
#### Part (i)

(3 points) Complete the function called `generate_bigM_c` to generate the big-M values by the method described in Question 5(a). Your function should take in as inputs:

- `A`, which is a $m \times n$ `numpy.ndarray`,
- `B`, which is a $n \times N$ `numpy.ndarray` where the $k$th column of `B` is $b^k$ as described in the pdf file, and
- `p`, the number of scenarios we wish the constraints to be held.

Your function should return a $m \times N$ `numpy.ndarray` where the $(i, k)$-th entry is $M_i^k$ (as defined on the pdf).

To help you test your functions, we have given you a sample input in the following cell. Note that the data may not be of the same dimension when we test your code, so make sure your code generalizes!

In [10]:
m = 50
n = 100
N = 30
np.random.seed(1729)
A = np.random.uniform(low=0, high=100, size=(m, n))
B = np.random.uniform(low=0, high=1000, size=(m, N))
c = np.random.uniform(low=0, high=100, size=n)
p = 15

In [11]:
def generate_bigM_c(A, B, p):
    M = None
    ###########################
    ### YOUR CODE HERE ########
    
    M = np.zeros((A.shape[0], B.shape[1]))
    for i in range(A.shape[0]):
        for k in range(B.shape[1]):
            M[i, k] = np.max(np.abs(A[i, :] * B[:, k]))
    return M

In [ ]:
# Assertion cell: Q5ci - Please to not edit or delete this cell

In [ ]:
# Assertion cell: Q5ci - Please to not edit or delete this cell

In [ ]:
# Assertion cell: Q5ci - Please to not edit or delete this cell

#### Part (ii)

(3 points) Complete the function called `solve_chance_constraints` that implements the model described in part (b). Your function should take as inputs the following:

- `A`, which is a $m \times n$ `numpy.ndarray`,
- `B`, which is a $n \times N$ `numpy.ndarray` where the $k$th column of `B` is $b^k$ as described in the pdf file,
- `c`, which is the cost vector stored as a 1-dimensional `numpy.ndarray` of length $n$,
- `p`, the number of scenarios we wish the constraints to be held, and
- `generate_bigM`, which is a function used to obtain the big-M values. Note that this is a generic function name. It is not the same as the function you wrote in part (i); however, when you run the code in part (d), you will use the function you wrote in part (i) as input.

Your function should return a tuple of the following:

- optimal objective value, 
- the value of $x$ at optimality,
- a 1-dimensional Boolean `numpy.ndarray` where the $k$th entry is `True` if the $k$th set of constraints are satisfied and `False` if otherwise, and
- the running time.

*Note.* You should only return the time taken to solve the MIP, i.e., only time taken by the `mod.optimize()` command. Do not include the time taken to construct the model. *Hint.* You can query the runtime by accessing a model parameter.

In [12]:
def solve_chance_constraints(A, B, c, p, generate_bigM):

    M = generate_bigM(A, B, p)

    mod = gp.Model('chance-constraints')
    mod.Params.OutputFlag = 0
    mod.Params.LogToConsole = 0

    opt_obj = None
    opt_x = None
    satisfied_constraints = None
    running_time = None
    ###########################
    ### YOUR CODE HERE ########
    
    start_time = time.time()
    x = mod.addVars(n, vtype=GRB.CONTINUOUS, name='x')
    mod.setObjective(c.T * x, GRB.MINIMIZE)
    for i in range(p):
        mod.addConstr(A * x <= M[i] * B[:, i])
    mod.optimize()
    running_time = time.time() - start_time
    opt_obj = mod.objVal
    opt_x = np.array([x[i].x for i in range(n)])
    satisfied_constraints = np.array([A * opt_x <= M[i] * B[:, i] for i in range(p)])
    return opt_obj, opt_x, satisfied_constraints, running_time

In [27]:
# Assertion cell: Q5cii - Please to not edit or delete this cell

In [28]:
# Assertion cell: Q5cii - Please to not edit or delete this cell

In [29]:
# Assertion cell: Q5cii - Please to not edit or delete this cell

### Q5(d)

Run the following cell to obtain the runtime. You may want to run this cell multiple times and average the results.

In [13]:
m = 50
n = 100
N = 30
np.random.seed(1729)
A = np.random.uniform(low=0, high=100, size=(m, n))
B = np.random.uniform(low=0, high=1000, size=(m, N))
c = np.random.uniform(low=0, high=100, size=n)
p = 15

chance_constraints = solve_chance_constraints(A, B, c, p, generate_bigM_c)
print(f'Running time: {chance_constraints[-1] :.4f}')

ValueError: ignored

### Q5(f)

(3 points) Complete the function called `generate_bigM_f` to generate the big-M values by the method described in Question 5(e). Your function should take in as inputs:

- `A`, which is a $m \times n$ `numpy.ndarray`,
- `B`, which is a $n \times N$ `numpy.ndarray` where the $k$th column of `B` is $b^k$ as described in the pdf file, and
- `p`, the number of scenarios we wish the constraints to be held.

Your function should return a $m \times N$ `numpy.ndarray` where the $(i, k)$-th entry is $\bar{M}_i^k$ (as defined on the pdf).

In [ ]:
def generate_bigM_f(A, B, p):
    M = None
    ###########################
    ### YOUR CODE HERE ########
  
    #Evaluate the mean of all the constraints:
    Ax_bar = A @ np.ones(n)
    #and the standard deviation of each constraint:
    Ax_sigma = np.power(A, 2) @ np.ones(n)
    Ax_sigma = np.sqrt(Ax_sigma - np.power(Ax_bar, 2))
    #Now, use Chebyshev's inequality to evaluate the maximum expected value of each constraint:
    Ax_max = Ax_bar + Ax_sigma
    
    #And now we can compute the bigM values as:
    M = Ax_max @ B.T
    return M


In [ ]:
# Assertion cell: Q5f - Please to not edit or delete this cell

In [ ]:
# Assertion cell: Q5f - Please to not edit or delete this cell

In [ ]:
# Assertion cell: Q5f - Please to not edit or delete this cell

### Q5(g)

Run the following cell to obtain the runtime for the new big-M values. You may want to run this cell multiple times and average the results.

In [ ]:
m = 50
n = 100
N = 30
np.random.seed(1729)
A = np.random.uniform(low=0, high=100, size=(m, n))
B = np.random.uniform(low=0, high=1000, size=(m, N))
c = np.random.uniform(low=0, high=100, size=n)
p = 15
chance_constraints = solve_chance_constraints(A, B, c, p, generate_bigM_f)
print(f'Running time: {chance_constraints[-1] :.4f}')